# Lab 3: Pipeline Interativo de Cálculo de Valor por Perfil de Pet

Este notebook executa o pipeline de cálculo de Valor por Perfil de Pet (LTV). As células abaixo irão compilar o código MapReduce em Java, ingerir dados do PostgreSQL com Sqoop, executar o job no Hadoop e verificar os resultados.

## 1. Compilando a Aplicação MapReduce

A primeira etapa é compilar nosso código-fonte Java em um arquivo `.jar` executável. A célula abaixo usa o Maven para isso. Ela navega até o diretório do projeto e executa o `mvn package`. A flag `-q` é para uma saída mais limpa.

In [ ]:
%%bash
cd mapreduce-jobs/ltv-by-pet-profile
mvn package -q
echo "JAR compilado com sucesso em: target/ltv-by-pet-profile-1.0-SNAPSHOT.jar"

## 2. Ingestão de Dados com Sqoop

Agora, vamos importar os dados de compras do PostgreSQL para o HDFS. O comando `hdfs dfs -rm` é usado para remover o diretório de destino antes da importação, garantindo que possamos executar esta célula várias vezes sem erros.

A query Sqoop já realiza um JOIN para obter o perfil do pet e o valor da compra, simplificando o MapReduce posterior.

In [ ]:
%%bash
INPUT_DIR=/petshop/input_ltv

# Remove o diretório de entrada se ele já existir
hdfs dfs -test -d $INPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS existente: $INPUT_DIR"
    hdfs dfs -rm -r $INPUT_DIR
fi

echo "Iniciando importação com Sqoop..."
sqoop import \
    --connect jdbc:postgresql://postgres:5432/petshop_db \
    --username petshop_user \
    --password petshop_password \
    --query "SELECT \
                CONCAT(p.especie, ';', p.tipo_pelo) AS perfil_pet, \
                (hc.quantidade * pr.preco) AS valor_compra \
            FROM  compra hc \
            JOIN produto pr ON hc.produto_id = pr.produto_id \
            JOIN pet p ON hc.tutor_id = p.tutor_id \
            WHERE \$CONDITIONS" \
    --target-dir $INPUT_DIR \
    --split-by hc.compra_id \
    --m 1

echo "\nImportação concluída. Verificando os dados no HDFS:"
hdfs dfs -ls $INPUT_DIR
hdfs dfs -cat $INPUT_DIR/part-m-00000 | head -n 5

## 3. Execução do Job MapReduce

Com os dados no HDFS, podemos executar nosso job MapReduce. A célula abaixo submete o `.jar` que compilamos para o Hadoop. O resultado será salvo no diretório `/petshop/output_ltv`.

In [ ]:
%%bash
INPUT_DIR=/petshop/input_ltv
OUTPUT_DIR=/petshop/output_ltv
JAR_PATH=/home/jovyan/work/labs/mapreduce-jobs/ltv-by-pet-profile/target/ltv-by-pet-profile-1.0-SNAPSHOT.jar

# Remove o diretório de saída se ele já existir
hdfs dfs -test -d $OUTPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS de saída existente: $OUTPUT_DIR"
    hdfs dfs -rm -r $OUTPUT_DIR
fi

echo "Executando o job MapReduce..."
hadoop jar $JAR_PATH com.petshop.hadoop.LTVCalculation $INPUT_DIR $OUTPUT_DIR

echo "\nJob concluído!"

## 4. Verificação do Resultado

Vamos verificar o resultado processado no HDFS. O arquivo `part-r-00000` deve conter o `perfil_pet` e o `valor_total`.

In [ ]:
%%bash
OUTPUT_DIR=/petshop/output_ltv
echo "Conteúdo do diretório de saída:"
hdfs dfs -ls $OUTPUT_DIR

echo "\nResultado do processamento:"
hdfs dfs -cat $OUTPUT_DIR/part-r-00000